# 改进模型
## 使用sklearn随机森林拟合非线性成分
- 用储存在**alg**的随机森林算法去做交叉验证。用predictions去预测Survived列。将结果赋值到**scores**。
- 使用**model_selection.cross_val_score**来完成这些。

In [5]:
import pandas as pd
titanic = pd.read_csv('Data/train.csv')
# 处理Age中的缺失值
titanic['Age'] = titanic['Age'].fillna(titanic['Age'].median())
# 量化Sex
titanic.loc[titanic['Sex'] == 'male', 'Sex'] = 0
titanic.loc[titanic['Sex'] == 'female', 'Sex'] = 1
# 量化Embarked
titanic['Embarked'] = titanic['Embarked'].fillna('S')
titanic.loc[titanic['Embarked'] == 'S', 'Embarked'] = 0
titanic.loc[titanic['Embarked'] == 'C', 'Embarked'] = 1
titanic.loc[titanic['Embarked'] == 'Q', 'Embarked'] = 2

In [7]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

predictors = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]

# Initialize our algorithm with the default paramters
# n_estimators is the number of trees we want to make
# min_samples_split is the minimum number of rows we need to make a split
# min_samples_leaf is the minimum number of samples we can have at the place where a tree branch ends (the bottom points of the tree)
alg = RandomForestClassifier(random_state=1, n_estimators=10, min_samples_split=2, min_samples_leaf=1)
# Compute the accuracy score for all the cross validation folds.  (much simpler than what we did before!)
scores = cross_val_score(alg, titanic[predictors], titanic["Survived"],cv=3)

# Take the mean of the scores (because we have one for each fold)
print(scores.mean())

0.801346801347


# 调参
- 增加我们使用的树的数量会很大的提升预测的准确率，训练更多的树会花费更多的时间
- 调整**min_samples_split**和**min_samples_leaf**变量来减少过拟合

In [8]:
alg = RandomForestClassifier(random_state=1, n_estimators=150, min_samples_split=4, min_samples_leaf=2)
# Compute the accuracy score for all the cross validation folds.  (much simpler than what we did before!)
scores = cross_val_score(alg, titanic[predictors], titanic["Survived"], cv=3)

# Take the mean of the scores (because we have one for each fold)
print(scores.mean())

0.820426487093


# 生成新特征
- 一些点子：
  - 名字的长度——这和那人有多富有，所以在泰坦尼克上的位置有关。
  - 一个家庭的总人数(**SibSp**+**Parch**)。
- 使用pandas数据框的**.apply**方法来生成特征。这会对你传入数据框(dataframe)或序列(series)的每一个元素应用一个函数。我们也可以传入一个**lambda**函数使我们能够定义一个匿名函数。
- 一个匿名的函数的语法是**lambda x:len(x)**。x将传入的值作为输入值——在本例中，就是乘客的名字。表达式的右边和返回结果将会应用于x。**.apply**方法读取这些所有输出并且用他们构造出一个pandas序列。我们可以将这个序列赋值到一个数据框列。